In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
test_df=pd.read_csv('./output/train_statement_section.csv')
test_df

,statement,section,label
0,All the primary trial participants do not rece...,"[""INTERVENTION 1:"", ""Diagnostic (FLT PET)"", ""P...",Contradiction
1,"Patients with Platelet count over 100,000/mm¬¨...","[""DISEASE CHARACTERISTICS:"", ""Histologically o...",Contradiction
2,Heart-related adverse events were recorded in ...,"[""Adverse Events 1:"", ""Total: 5/32 (15.63%)"", ...",Entailment
3,Adult Patients with histologic confirmation of...,"[""Inclusion Criteria:"", ""Patients with histolo...",Contradiction
4,Laser Therapy is in each cohort of the primary...,"[""INTERVENTION 1:"", ""Laser Therapy Alone"", ""th...",Contradiction
...,...,...,...
1695,"Adequate blood, kidney, and hepatic function a...","[""Inclusion Criteria:"", ""Postmenopausal women,...",Entailment
1696,The Ridaforolimus + Dalotuzumab + Exemestane g...,"[""Outcome Measurement:"", ""1. Progression-free ...",Contradiction
1697,The only difference between the interventions ...,"[""INTERVENTION 1:"", ""Prone"", ""Prone position"",...",Entailment
1698,Patients must have a white blood cell count ab...,"[""DISEASE CHARACTERISTICS:"", ""Histologically c...",Entailment


In [3]:
hypothesis_lst=test_df['statement'].values.tolist()
len(hypothesis_lst)

1700

In [4]:
evidence_lst=test_df['section'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

1700

In [5]:
label2id={"Contradiction":0,"Entailment":1}
label_lst=test_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

1700

In [6]:
import json
with open('output/all_evidences.json','r') as f:
    all_evidences = json.load(f)
evidence2id=dict([(all_evidences[i],i) for i in range(len(all_evidences))])

from nltk.tokenize import word_tokenize
tokenized_corpus = [word_tokenize(doc) for doc in all_evidences]

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(tokenized_corpus)

In [7]:
import numpy as np
scores=[]
for hypothesis,evidences in test_df[['statement','section']].values:
    doc_scores = bm25.get_scores(word_tokenize(hypothesis))
    doc_scores = (doc_scores-np.min(doc_scores))/(np.max(doc_scores)-np.min(doc_scores))
    scores.append(np.mean([doc_scores[evidence2id[evi]] for evi in json.loads(evidences)]))
# scores

In [8]:
from sklearn.metrics import average_precision_score,f1_score,precision_score,recall_score,accuracy_score

y_true=label_lst
results=[]
for thre in np.arange(0.0, 1.0, 0.1):
    y_prob=scores
    y_pred=[1 if a>thre else 0 for a in y_prob]
    results.append([
        thre,
        average_precision_score(y_true,y_prob),
        f1_score(y_true,y_pred),
        precision_score(y_true,y_pred),
        recall_score(y_true,y_pred),
        accuracy_score(y_true,y_pred)
    ])

import pandas as pd

pd.DataFrame(results,columns=['threshold','AVG_PREC','F1','PREC','REC','ACC'])

/home/ai/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,threshold,AVG_PREC,F1,PREC,REC,ACC
0,0.0,0.489528,0.635452,0.492866,0.894118,0.487059
1,0.1,0.489528,0.469815,0.477521,0.462353,0.478235
2,0.2,0.489528,0.243556,0.498182,0.161176,0.499412
3,0.3,0.489528,0.095643,0.494505,0.052941,0.499412
4,0.4,0.489528,0.042649,0.463415,0.022353,0.498235
5,0.5,0.489528,0.013921,0.500000,0.007059,0.500000
6,0.6,0.489528,0.007001,0.428571,0.003529,0.499412
7,0.7,0.489528,0.000000,0.000000,0.000000,0.500000
8,0.8,0.489528,0.000000,0.000000,0.000000,0.500000
9,0.9,0.489528,0.000000,0.000000,0.000000,0.500000
